---
**License**

 NumCosmoCCLTest

 Mon Jun 01 09:20:00 2020\
 Copyright  2020\
 Sandro Dias Pinto Vitenti <vitenti@uel.br>

---
---

 NumCosmoCCLTest\
 Copyright (C) 2020 Sandro Dias Pinto Vitenti <vitenti@uel.br>

 numcosmo is free software: you can redistribute it and/or modify it
 under the terms of the GNU General Public License as published by the
 Free Software Foundation, either version 3 of the License, or
 (at your option) any later version.

 numcosmo is distributed in the hope that it will be useful, but
 WITHOUT ANY WARRANTY; without even the implied warranty of
 MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.
 See the GNU General Public License for more details.

 You should have received a copy of the GNU General Public License along
 with this program.  If not, see <http://www.gnu.org/licenses/>.
 
---

In [ ]:
try:
    import gi

    gi.require_version("NumCosmo", "1.0")
    gi.require_version("NumCosmoMath", "1.0")
except:
    pass

from gi.repository import GObject
from gi.repository import NumCosmo as Nc
from gi.repository import NumCosmoMath as Ncm

import pyccl as ccl

import sys
import numpy as np
import math
import matplotlib.pyplot as plt

import sys

sys.path.insert(0, "../scripts")

from nc_ccl import create_nc_obj, ccl_cosmo_set_high_prec, dsigmaM_dlnM
from notebook_helpers import compare_ccl_nc_func

In [ ]:
Ncm.cfg_init()
Ncm.cfg_set_log_handler(lambda msg: sys.stdout.write(msg) and sys.stdout.flush())

In [ ]:
Omega_c = 0.25
Omega_b = 0.05
Omega_k = 0.0
h = 0.7
A_s = 2.1e-9
n_s = 0.96
Neff = 0.0
sigma8 = 0.9
mu_0 = 0.1
sigma_0 = 0.1

Omega_v_vals = np.array([0.7, 0.7, 0.7, 0.65, 0.75])
w0_vals = np.array([-1.0, -0.9, -0.9, -0.9, -0.9])
wa_vals = np.array([0.0, 0.0, 0.1, 0.1, 0.1])

mnu = [[0.04, 0.0, 0.0], [0.05, 0.01, 0.0], [0.05, 0.0, 0.0], [0.03, 0.02, 0.0]]
Neff_mnu = 3.0

In [ ]:
def compare_distances(z, Omega_v, w0, wa):
    """
    Compare distances calculated by pyccl with the distances in the benchmark
    file.
    This test is only valid when radiation is explicitly set to 0.
    """
    # Set Omega_K in a consistent way
    Omega_k = 1.0 - Omega_c - Omega_b - Omega_v

    ccl_cosmo = ccl.Cosmology(
        Omega_c=Omega_c,
        Omega_b=Omega_b,
        Neff=Neff,
        h=h,
        A_s=A_s,
        n_s=n_s,
        Omega_k=Omega_k,
        w0=w0,
        wa=wa,
        transfer_function="eisenstein_hu",
    )
    ccl_cosmo_set_high_prec(ccl_cosmo)

    cosmo, dist, ps_lin, ps_nln, hmfunc = create_nc_obj(ccl_cosmo)

    a = 1.0 / (1.0 + z)

    ccl_Og = ccl.omega_x(ccl_cosmo, a, "matter")
    nc_Og = [cosmo.E2Omega_m(z_i) / cosmo.E2(z_i) for z_i in z]

    compare_ccl_nc_func(z, ccl_Og, nc_Og, x_name="z", y_name="\Omega_m")

    ccl_E = ccl.h_over_h0(ccl_cosmo, a)
    nc_E = [cosmo.E(z_i) for z_i in z]

    compare_ccl_nc_func(z, ccl_E, nc_E, x_name="z", y_name="E")

    ccl_chi = ccl.comoving_radial_distance(ccl_cosmo, a)
    RH_Mpc = cosmo.RH_Mpc()
    nc_chi = [dist.comoving(cosmo, z_i) * RH_Mpc for z_i in z]

    compare_ccl_nc_func(z, ccl_chi, nc_chi, x_name="z", y_name="D_c")

In [ ]:
z = np.linspace(0.0, 15.0, 10000)

for i in range(5):
    compare_distances(z, Omega_v_vals[i], w0_vals[i], wa_vals[i])

In [ ]:
def compare_growth(z, Omega_v, w0, wa):
    """
    Compare ...
    """

    # Set Omega_K in a consistent way
    Omega_k = 1.0 - Omega_c - Omega_b - Omega_v

    ccl_cosmo = ccl.Cosmology(
        Omega_c=Omega_c,
        Omega_b=Omega_b,
        Neff=Neff,
        h=h,
        A_s=A_s,
        n_s=n_s,
        Omega_k=Omega_k,
        w0=w0,
        wa=wa,
        transfer_function="eisenstein_hu",
    )
    ccl_cosmo_set_high_prec(ccl_cosmo)

    cosmo, dist, ps_lin, ps_nln, hmfunc = create_nc_obj(ccl_cosmo)

    gf = ps_lin.peek_gf()
    gf.prepare(cosmo)

    a = 1.0 / (1.0 + z)

    nc_D_z = [gf.eval(cosmo, z_i) for z_i in z]
    ccl_D_z = ccl.growth_factor(ccl_cosmo, a)

    compare_ccl_nc_func(z, ccl_D_z, nc_D_z, x_name="z", y_name="D", xscale="linear")

    nc_f_z = [
        -(1.0 + z_i) / gf.eval(cosmo, z_i) * gf.eval_deriv(cosmo, z_i) for z_i in z
    ]
    ccl_f_z = ccl.growth_rate(ccl_cosmo, a)

    compare_ccl_nc_func(z, ccl_f_z, nc_f_z, x_name="z", y_name="f", xscale="linear")

In [ ]:
z = np.linspace(0.0, 5.0, 10000)

compare_growth(z, 0.7, -1.0, 0.0)

In [ ]:
def compare_ps(k, z, Omega_v, w0, wa):
    """
    Compare ...
    """

    # Set Omega_K in a consistent way
    Omega_k = 1.0 - Omega_c - Omega_b - Omega_v

    ccl_cosmo = ccl.Cosmology(
        Omega_c=Omega_c,
        Omega_b=Omega_b,
        Neff=Neff,
        h=h,
        sigma8=sigma8,
        n_s=n_s,
        Omega_k=Omega_k,
        w0=w0,
        wa=wa,
        transfer_function="eisenstein_hu",
    )
    ccl_cosmo_set_high_prec(ccl_cosmo)

    cosmo, dist, ps_lin, ps_nln, hmfunc = create_nc_obj(ccl_cosmo)

    for z_i in z:
        a_i = 1.0 / (1.0 + z_i)
        pk_ccl = ccl.linear_matter_power(ccl_cosmo, k, a_i)
        pk_nc = np.array([ps_lin.eval(cosmo, z_i, k_i) for k_i in k])
        compare_ccl_nc_func(
            k, pk_ccl, pk_nc, x_name="k", y_name="P_k(%f)" % z_i, xscale="log"
        )

In [ ]:
k = np.geomspace(1.0e-6, 1.0e3, 1000)
z = [0.0, 0.5, 1.0]
compare_ps(k, z, 0.7, -1.0, 0.0)

In [ ]:
def compare_sigmaR(R, z, Omega_v, w0, wa):
    """
    Compare ...
    """

    # Set Omega_K in a consistent way
    Omega_k = 1.0 - Omega_c - Omega_b - Omega_v

    ccl_cosmo = ccl.Cosmology(
        Omega_c=Omega_c,
        Omega_b=Omega_b,
        Neff=Neff,
        h=h,
        sigma8=sigma8,
        n_s=n_s,
        Omega_k=Omega_k,
        w0=w0,
        wa=wa,
        transfer_function="eisenstein_hu",
    )
    ccl_cosmo_set_high_prec(ccl_cosmo)

    cosmo, dist, ps_lin, ps_nln, hmfunc = create_nc_obj(ccl_cosmo)
    psf = hmfunc.peek_psf()

    for z_i in z:
        a_i = 1.0 / (1.0 + z_i)
        sigma_ccl = ccl.sigmaR(ccl_cosmo, R, a_i)
        sigma_nc = np.array([psf.eval_sigma(z_i, R_i) for R_i in R])
        compare_ccl_nc_func(
            R, sigma_ccl, sigma_nc, x_name="R", y_name=r"\sigma(%f)" % z_i, xscale="log"
        )

In [ ]:
# Array of radius
R = np.geomspace(1.0e-1, 1.0e2, 1000)

# Array of redshifts
z = np.linspace(0.0, 1.0, 3)

compare_sigmaR(R, z, 0.7, -1.0, 0.0)

In [ ]:
def compare_sigmaM(M, z, Omega_v, w0, wa):
    """
    Compare ...
    """

    # Set Omega_K in a consistent way
    Omega_k = 1.0 - Omega_c - Omega_b - Omega_v

    ccl_cosmo = ccl.Cosmology(
        Omega_c=Omega_c,
        Omega_b=Omega_b,
        Neff=Neff,
        h=h,
        sigma8=sigma8,
        n_s=n_s,
        Omega_k=Omega_k,
        w0=w0,
        wa=wa,
        transfer_function="eisenstein_hu",
    )
    ccl_cosmo_set_high_prec(ccl_cosmo)

    cosmo, dist, ps_lin, ps_nln, hmfunc = create_nc_obj(ccl_cosmo)
    psf = hmfunc.peek_psf()

    for z_i in z:
        a_i = 1.0 / (1.0 + z_i)

        sigma_ccl = ccl.sigmaM(ccl_cosmo, M, a_i)
        sigma_nc = np.array(
            [
                psf.eval_sigma(z_i, math.exp(hmfunc.lnM_to_lnR(cosmo, math.log(M_i))))
                for M_i in M
            ]
        )
        compare_ccl_nc_func(
            M,
            sigma_ccl,
            sigma_nc,
            x_name="M",
            y_name=r"\sigma(%.1f)" % z_i,
            xscale="log",
        )

        dsigma_ccl = dsigmaM_dlnM(ccl_cosmo, M, a_i)
        dsigma_nc = np.array(
            [
                (-math.log(10) / 6.0)
                * psf.eval_dlnvar_dlnr(z_i, hmfunc.lnM_to_lnR(cosmo, math.log(M_i)))
                for M_i in M
            ]
        )
        compare_ccl_nc_func(
            M,
            dsigma_ccl,
            dsigma_nc,
            x_name="M",
            y_name=r"\frac{d\ln\sigma}{d\log_{10} M}(%.1f)" % z_i,
            xscale="log",
        )

In [ ]:
# Array of masses
M = np.geomspace(1.0e10, 1.0e14, 1000)

# Array of redshifts
z = np.linspace(0.0, 1.0, 3)

compare_sigmaM(M, z, 0.7, -1.0, 0.0)